In [ ]:
#need 2 models
    #1.  Predict completely new products -373 prods generating 15k submissions
    #2.  Predict sales for those that have sales in the past, but not in certain shops
#This EDA focuses on the first element

In [2]:
#start with the end in mind - 
#what information do I have about the entirely new products
    #prices
    #categories
    #descriptions in the names
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np
import os
import datetime as dt
#read in test data set
test=pd.read_csv("/home/cdsw/01_future-sales/data/test.csv")
#read in train data set
dateCols = ['date']
train2=pd.read_csv("/home/cdsw/train_plus_not_imputted.csv",parse_dates=dateCols)
train2["TotalRevenue"]=train2.item_cnt_day*train2.item_price
train2["Month"]=train2.date.dt.month
train2["Year"]=train2.date.dt.year
train2['YM'] = train2['Year']*100+train2["Month"]

#restrict train to pre 1/11/15 - take test where prod not in
train_pre=train2[train2.date<="2015-10-31"]

test2=test[~test.item_id.isin(train_pre.item_id)]
print("Number of observations to predict:",test2.shape)
print("Number of products to predict:", len(set(test2.item_id)))

Number of observations to predict: (15246, 3)
Number of products to predict: 363


In [3]:
#check if any of these products have sales or price data later than November?
train_post=train2[train2.date>"01-11-2015"]
sneekpeak=train_post[train_post.item_id.isin(test2.item_id)]
sneekpeak.shape

(0, 13)

In [9]:
#so zero information here - no price points, no availability of stock, no product names...
#what are their categories...
cats= pd.read_csv("/home/cdsw/01_future-sales/data/item_categories.csv")
cats.head()
#the category information would require a cat id which doesn't exist 
#therefore, we need to look at all those products that weren't sold previously and see how their sales progressed...
    #are their main shops that they tend to sell in  - how to predict this?

,ID,shop_id,item_id
0,0,5,5037
1,1,5,5320
2,2,5,5233
3,3,5,5232
4,4,5,5268


In [10]:
#create a dataset which has only new products in it
sales_per_item_PM= train2.groupby(['item_id', "shop_id",'YM'])['item_cnt_day',"TotalRevenue" ].sum().reset_index()
sales_per_item_PM["AvPrice"]=sales_per_item_PM.TotalRevenue/sales_per_item_PM.item_cnt_day
sales_per_item_PM["id"]=sales_per_item_PM.item_id.astype("str")+"_"+sales_per_item_PM.shop_id.astype("str")
sales_per_item_PM["prod_month"]=sales_per_item_PM.YM.astype("str")+"_"+sales_per_item_PM.item_id.astype("str")


#create a dataset which is sorted by date
firstsale=train2.groupby(["YM","date", "item_id"])["item_cnt_day"].sum().reset_index().sort_values("date")
#drop_duplicates
firstsale2=firstsale.drop_duplicates(subset=["item_id"],keep="first").reset_index()
firstsale2["prod_month"] = firstsale2.YM.astype("str")+"_"+firstsale2.item_id.astype("str")
firstsale2["present"]=1
firsts=pd.merge(sales_per_item_PM, firstsale2[["prod_month","present"]],how="left",on="prod_month")
#those where present is null, are not the first date, remove them
firsts2=firsts[firsts.present.notnull()]
#many products will have been sold before hand and so because of the start of Jan 2013, this is an artifical start
firsts3=firsts2[firsts2.YM>201301]

#need to find out how many days it was on sale for  - and what was the first day it was sold - was it towards the start of the month?
train2["prod_month"]=train2.YM.astype("str")+"_"+train2.item_id.astype("str")
train3=pd.merge(train2, firstsale2[["prod_month","present"]], how="left",on="prod_month")

#non nulls only -then need to sort by item and days and keep first - because have shop info here meaning dates are duplicated
train4=train3[train3.present.notnull()].sort_values(["item_id","date"])
train5=train4.drop_duplicates(subset=["item_id","date"],keep="first")

#then group by
Numsales=train5.groupby(["item_id"])["date"].count().reset_index(name="NumDays_1stM")

#identify the day where the first sale occured 
firstdate=train5.drop_duplicates(["item_id"],keep="first")
firstdate["Day"]=firstdate["date"].dt.day

Numsales2=pd.merge(Numsales, firstdate[["item_id","YM","Day"]], how="left", on="item_id")


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [18]:
#firsts, are the first month
# firsts3.groupby(["YM"])
prods=firsts3.drop_duplicates("item_id",keep="first")
prods.groupby(["YM"])["item_id"].count().reset_index()

,YM,item_id
0,201302,1610
1,201303,753
2,201304,411
3,201305,457
4,201306,313
5,201307,292
6,201308,235
7,201309,162
8,201310,239
9,201311,186


In [ ]:

#Numsales3=Numsales2.pivot_table(index=["Day"],columns="NumDay_1stM")
#Numsales3=Numsales2.groupby(["Day","NumDays_1stM"])["Day"].count().reset_index(name="Freq")
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(pd.crosstab(Numsales2['Day'],Numsales2['NumDays_1stM']))

In [ ]:
#are there new product sales in October that I can look at to compare?
#started doing some of this in EDA5

#do new products always start at the same price?  is there variation - no information on price

#could build a data set per shop - have the features as the previous months of new sales per product
    #likely to have variation in the range of sales... but
    #what other information do I have to create a prediction
    #would mean predict the same for every new product in that shop
    #create the data set and see what the variation in data is....
    #will you have enough data to do this
        #there are 60 shops - these are your rows...how can you split out a test set from here
#how many new products are there -11k - can push out a couple more months  - but could even be 8k to be safe
            # could do new products sold, 
                #feature being sales of new products prior months - 
                #sales of existing prods
                #variability of price in old, new
                    #which are effectively identifying the demand for products
            #this would be product based - need to get it to shop level - how many shops are sold for new prods - 
                #on average 3 - are they always the same?

In [28]:
shp=firsts3.groupby(["shop_id"])["YM"].count().reset_index(name="CountofFirstShops").sort_values("CountofFirstShops",ascending=False).reset_index()
shp["Prop"]=round(shp.CountofFirstShops/sum(shp.CountofFirstShops),2)*100
shp["CumProp"]=shp.Prop.cumsum()
shp["Order"]=shp.index
shp

,index,shop_id,CountofFirstShops,Prop,CumProp,Order
0,25,25,2408,7.0,7.0,0
1,31,31,2148,6.0,13.0,1
2,55,55,1841,6.0,19.0,2
3,54,54,1776,5.0,24.0,3
4,28,28,1309,4.0,28.0,4
5,42,42,1160,3.0,31.0,5
6,6,6,1110,3.0,34.0,6
7,57,57,1005,3.0,37.0,7
8,27,27,996,3.0,40.0,8
9,21,21,801,2.0,42.0,9


In [44]:
#OK so not just a few shops.  I can try to predict volumes and take these predictions and push it forward into shops
    # but this will be the same for every product 
# set(test2.item_id)
len(test2.shop_id[test2.item_id==7607])

42

In [46]:
firsts3.tail()

,item_id,shop_id,YM,item_cnt_day,TotalRevenue,AvPrice,id,prod_month,present
1798781,22164,47,201501,1.0,749.0,749.0,22164_47,201501_22164,1.0
1798795,22164,52,201501,1.0,549.0,549.0,22164_52,201501_22164,1.0
1798833,22165,55,201308,1.0,59.0,59.0,22165_55,201308_22165,1.0
1798835,22166,55,201302,5.0,750.0,150.0,22166_55,201302_22166,1.0
1799394,22169,25,201403,1.0,4349.0,4349.0,22169_25,201403_22169,1.0
